# Librerías

In [1]:
# cálculos
import numpy as np
import pandas as pd
import math

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# modelos
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# metricas
import sklearn.metrics as metrics

# hiperparametrizado
from sklearn.model_selection import GridSearchCV

# quitar alertas
import warnings
warnings.filterwarnings("ignore")



In [2]:
 # Ruta al archivo 
ruta_archivo = "C:/Users/Herre/Desktop/Entrega_TFM/TFM_EasyMoney/modelo_account.csv"
# Leer el archivo 
df_modelo = pd.read_csv(ruta_archivo)

In [5]:
# Separar las características (X) y la variable objetivo (y)
X = df_modelo.drop(columns=['category_accounts'])  # Todas las columnas excepto 'category_accounts'
y = df_modelo['category_accounts']  # La columna 'category_accounts' es la variable objetivo

In [6]:
# Dividir los datos (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Toreno de modelos

In [7]:
f1s = pd.DataFrame(columns=['modelo', 'f1_score'])
f1s

,modelo,f1_score


In [8]:
# añadimos en una lista los modelos que queremos poner a competir

modelos = []
modelos.append(('LogisticRegression', LogisticRegression()))
modelos.append(('RidgeClassifier', RidgeClassifier()))
modelos.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
modelos.append(('RandomForestClassifier', RandomForestClassifier()))
modelos.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
modelos.append(('XGBClassifier', XGBClassifier()))
modelos.append(('CatBoostClassifier', CatBoostClassifier(verbose=False)))

In [9]:
for nombre, modelo in modelos:
  # entrenamiento
  modelo.fit(X_train, y_train)

  # predicción del test
  y_pred = modelo.predict(X_test)

  # evalua la metrica
  f1_score = metrics.f1_score(y_test, y_pred, average='macro')
  #el average='macro' se usa cuando estamos haciendo modelos multiclases. Sino lo pones el modelo peta.

  # añade la métrica al dataframe de métricas
  metrica = pd.DataFrame({'modelo':[nombre], 'f1_score':[f1_score]})

  print(nombre, np.round(f1_score,5))

  f1s = pd.concat([f1s,metrica], ignore_index=True)

LogisticRegression 0.57774
RidgeClassifier 0.75345
DecisionTreeClassifier 0.83095
RandomForestClassifier 0.87576
GradientBoostingClassifier 0.83608
XGBClassifier 0.86195
CatBoostClassifier 0.86234


In [10]:
f1s.sort_values('f1_score', ascending= False)

,modelo,f1_score
3,RandomForestClassifier,0.875765
6,CatBoostClassifier,0.862337
5,XGBClassifier,0.861949
4,GradientBoostingClassifier,0.836081
2,DecisionTreeClassifier,0.830953
1,RidgeClassifier,0.753447
0,LogisticRegression,0.577745


# Modelo

In [11]:
# Ajuste de hiperparámetros
xgb_model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')

In [12]:
# Entrenar el modelo con los datos de entrenamiento
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [13]:
# Hacer predicciones con el conjunto de prueba
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probabilidad de la clase 1 (compra)

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión: {accuracy * 100:.2f}%")

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(conf_matrix)

# Mostrar el reporte de clasificación
class_report = classification_report(y_test, y_pred)
print("Reporte de clasificación:")
print(class_report)

Precisión: 83.63%
Matriz de confusión:
[[19338  4055]
 [ 3650 20036]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.83      0.83     23393
           1       0.83      0.85      0.84     23686

    accuracy                           0.84     47079
   macro avg       0.84      0.84      0.84     47079
weighted avg       0.84      0.84      0.84     47079



In [15]:
y_pred

array([0, 1, 1, ..., 0, 0, 1])

In [16]:
y_pred_proba

array([0.18857718, 0.9509978 , 0.79444826, ..., 0.18173477, 0.23596923,
       0.93935233], dtype=float32)

In [17]:
# Crear un nuevo DataFrame con 'pk_cid', las predicciones y las probabilidades

df_resultado = X_test[['pk_cid']].copy()  # Copiar la columna 'pk_cid' de X_test
df_resultado['prediccion'] = y_pred  # Agregar las predicciones
df_resultado['probabilidad_compra'] = y_pred_proba  # Agregar la probabilidad de compra

# Mostrar el nuevo DataFrame
print(df_resultado.head())  # Muestra las primeras filas del DataFrame

         pk_cid  prediccion  probabilidad_compra
195505  1315049           0             0.188577
52639   1236786           1             0.950998
8639    1245634           1             0.794448
73121   1039443           0             0.492112
110786  1336676           0             0.456566


In [18]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47079 entries, 195505 to 104496
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_cid               47079 non-null  int64  
 1   prediccion           47079 non-null  int32  
 2   probabilidad_compra  47079 non-null  float32
dtypes: float32(1), int32(1), int64(1)
memory usage: 1.1 MB


En este dataframe tenemos muchos clientes, por lo tanto vamos a filtrar por aquellos que tienen un probabilidad de compra de casi el 100%

In [20]:
# Filtrar filas donde la probabilidad de compra sea mayor al 96.5%
df_filtrado = df_resultado[df_resultado['probabilidad_compra'] > 0.965]

In [22]:
df_filtrado

,pk_cid,prediccion,probabilidad_compra
207652,1477908,1,0.976016
229080,1281863,1,0.977176
127715,1453248,1,0.980992
73373,1411222,1,0.970630
74331,1409959,1,0.981902
...,...,...,...
83686,1430384,1,0.980992
97732,1028778,1,0.980614
129448,1139745,1,0.968910
14283,1465129,1,0.967626


Para el siguiente paso del proyecto nos piden 10 mil clientes por lo tanto no podemos coger a todos los clientes, cogemos el número de clientes que nos faltan para llegar a 10 mil.

In [23]:
# Ordenar el dataframe por la columna 'probabilidad_compra' en orden descendente
df_ordenado = df_filtrado.sort_values(by='probabilidad_compra', ascending=False)

# Seleccionar las primeras 4000 filas
df_top_4000 = df_ordenado.head(4076)

In [24]:
df_top_4000

,pk_cid,prediccion,probabilidad_compra
36844,1414328,1,0.987442
101987,1413873,1,0.987442
85626,1417700,1,0.987442
105398,1417966,1,0.987435
104241,1418254,1,0.987435
...,...,...,...
45034,1181335,1,0.968026
202033,1167703,1,0.968026
183627,1182728,1,0.968026
125028,1163527,1,0.968026


Exportamos la prediccion porque la necesitaremos más adelante para conocer los ingresos que tendrá la empresa

In [27]:
# Guardar el DataFrame en un archivo CSV para revisarlo luego
df_top_4000.to_csv('accounts_resultado_predicciones.csv', index=False)